In [1]:
%pip install datasets matplotlib underthesea gensim streamlit

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/560.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/560.4 kB ? eta -:--:--
     ------------------ --------------------- 262.1/560.4 kB ? eta -:--:--
     -------------------------------------- 560.4/560.4 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'


ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\pc\anaconda3\envs\aka_env\Lib\site-packages\pip\_internal\cli\base_command.py", line 106, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "c:\Users\pc\anaconda3\envs\aka_env\Lib\site-packages\pip\_internal\cli\base_command.py", line 97, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pc\anaconda3\envs\aka_env\Lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pc\anaconda3\envs\aka_env\Lib\site-packages\pip\_internal\commands\install.py", line 386, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "c:\Users\pc\anaconda3\envs\aka_env\Lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                     

### Import

In [ ]:
import time
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from gensim.models import Word2Vec
import multiprocessing
import joblib
import logging
import os
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


### Gộp data 

In [ ]:
class DataProcessor:
    def __init__(self, output_dir="./processed_data/"):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
        # Mapping chuẩn cho sentiment
        self.sentiment_mapping = {
            "negative": 0, 
            "neutral": 1, 
            "positive": 2,
            # Mapping ngược
            0: 0, 1: 1, 2: 2,
            # Biến thể
            "Negative": 0, "Neutral": 1, "Positive": 2,
            "NEGATIVE": 0, "NEUTRAL": 1, "POSITIVE": 2
        }

    def normalize_sentiment(self, value):
        if pd.isna(value):
            return None
            
        if isinstance(value, (int, float)):
            return int(value) if value in [0, 1, 2] else None
            
        if isinstance(value, str):
            value = value.lower().strip()
            return self.sentiment_mapping.get(value)
            
        return None

    def validate_data(self, df, name=""):
        logging.info(f"\nValidating {name}:")
        logging.info(f"Total samples: {len(df)}")
        logging.info(f"Missing values: {df.isnull().sum().to_dict()}")
        
        sentiment_dist = df['sentiment'].value_counts()
        logging.info("\nSentiment distribution:")
        for sent, count in sentiment_dist.items():
            logging.info(f"Sentiment {sent}: {count} ({count/len(df)*100:.2f}%)")

    def load_and_process_data(self, is_synthetic=False, **paths):
        try:
            if is_synthetic:
                df = pd.read_csv(paths['path'])
                df = df[['sentence', 'sentiment', 'topic']].copy()
            else:
                ds = load_dataset("uitnlp/vietnamese_students_feedback")
                df = ds[paths['split']].to_pandas()
                df = df.rename(columns={'text': 'sentence', 'label': 'sentiment'})
                df['topic'] = 'general'

            # Chuẩn hóa sentiment
            df['sentiment'] = df['sentiment'].apply(self.normalize_sentiment)
            df = df.dropna(subset=['sentiment'])
            df['sentiment'] = df['sentiment'].astype(int)

            return df

        except Exception as e:
            logging.error(f"Error processing data: {str(e)}")
            return None

    def merge_and_save_data(self):
        """Xử lý và gộp dữ liệu"""
        dfs = {
            'train': self.load_and_process_data(split='train'),
            'val': self.load_and_process_data(split='validation'),
            'test': self.load_and_process_data(split='test')
        }
        
        # Load dữ liệu synthetic
        synthetic = {
            'train': self.load_and_process_data(
                is_synthetic=True, 
                path="./chatgpt_data/synthetic_train.csv"
            ),
            'val': self.load_and_process_data(
                is_synthetic=True,
                path="./chatgpt_data/synthetic_val.csv"
            )
        }

        if any(df is None for df in dfs.values()) or any(df is None for df in synthetic.values()):
            return

        # Gộp và xử lý
        combined = {
            'train': pd.concat([dfs['train'], synthetic['train']], ignore_index=True),
            'val': pd.concat([dfs['val'], synthetic['val']], ignore_index=True),
            'test': dfs['test']
        }

        for split in ['train', 'val']:
            combined[split] = combined[split].drop_duplicates(subset='sentence')
            self.validate_data(combined[split], f"{split} set")

        for split, df in combined.items():
            df.to_csv(f"{self.output_dir}{split}.csv", index=False)

        self.plot_distributions(combined)
        
        return combined

    def plot_distributions(self, dfs):
        """Vẽ biểu đồ phân bố"""
        plt.figure(figsize=(10, 6))
        counts = {split: len(df) for split, df in dfs.items()}
        plt.bar(counts.keys(), counts.values())
        plt.title("Phân bố số lượng mẫu")
        for i, (split, count) in enumerate(counts.items()):
            plt.text(i, count, str(count), ha='center', va='bottom')
        plt.savefig(f"{self.output_dir}sample_distribution.png")
        plt.close()

        plt.figure(figsize=(10, 8))
        train_sentiments = dfs['train']['sentiment'].value_counts()
        colors = ['#ff9999', '#66b3ff', '#99ff99']
        plt.pie(
            train_sentiments,
            labels=[f"Sentiment {i}" for i in range(3)],
            colors=colors,
            autopct='%1.1f%%'
        )
        plt.title("Phân bố sentiment trong tập train")
        plt.savefig(f"{self.output_dir}sentiment_distribution.png")
        plt.close()

if __name__ == "__main__":
    processor = DataProcessor()
    data = processor.merge_and_save_data()

In [ ]:
# Đọc dữ liệu từ thư mục mới
new_data_dir = "processed_data"

# Đọc các tệp CSV
new_train = pd.read_csv(os.path.join(new_data_dir, "train.csv"))
new_val = pd.read_csv(os.path.join(new_data_dir, "val.csv"))
new_test = pd.read_csv(os.path.join(new_data_dir, "test.csv"))
# Hiển thị một vài dòng dữ liệu mới
print(new_train.head())
print(new_val.head())
print(new_test.head())

In [ ]:
print("10 samples from new_train with sentiment = 0:")
print(new_train[new_train['sentiment'] == 0].head(5))


In [ ]:
print("10 samples from new_train with sentiment = 1:")
print(new_train[new_train['sentiment'] == 1].head(5))


In [ ]:
print("10 samples from new_train with sentiment = 1:")
print(new_train[new_train['sentiment'] == 2].head(5))


In [ ]:
label_counts = new_train['sentiment'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(label_counts, labels=label_counts.index, autopct='%1.1f%%', startangle=140)
plt.title("Phân bố các nhãn trong tập huấn luyện")
plt.show()


In [ ]:
import os
import pandas as pd
import re
import unicodedata
from underthesea import word_tokenize

# Đọc dữ liệu từ thư mục mới
new_data_dir = "processed_data"

# Đọc các tệp CSV
new_train = pd.read_csv(os.path.join(new_data_dir, "train.csv"))
new_val = pd.read_csv(os.path.join(new_data_dir, "val.csv"))
new_test = pd.read_csv(os.path.join(new_data_dir, "test.csv"))

# Từ điển viết tắt cần thay thế
TEENCODE_DICT = {
    'ko': 'không', 'k': 'không', 'kh': 'không', 'kg': 'không',
    'dc': 'được', 'vs': 'với', 'nx': 'những', 'tk': 'tài khoản',
    'r': 'rồi', 'wa': 'quá', 'ht': 'hiện tại'
}

# Xử lý dữ liệu
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Chuẩn hóa Unicode & chuyển về chữ thường
    text = unicodedata.normalize('NFKC', text.lower())

    # Loại bỏ HTML & URLs
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Loại bỏ emoji
    text = re.sub(r"["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        "]+", '', text, flags=re.UNICODE)

    # Loại bỏ ký tự đặc biệt
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Thay thế viết tắt bằng từ đầy đủ
    text = ' '.join([TEENCODE_DICT.get(word.lower(), word) for word in text.split()])

    # Tách từ tiếng Việt
    text = word_tokenize(text, format="text")

    return text

# Xử lý từng tập dữ liệu và lưu kết quả
for name, df in [("train", new_train), ("val", new_val), ("test", new_test)]:
    print(f"Đang xử lý {name}...")

    # Xử lý tất cả văn bản
    df["processed_text"] = df["sentence"].apply(preprocess_text)

    # Lưu kết quả
    output_path = os.path.join(new_data_dir, f"{name}_processed.csv")
    df.to_csv(output_path, index=False)

    print(f"✔ Đã lưu: {output_path}")

    # In ví dụ kết quả xử lý
    print("\nVí dụ kết quả xử lý:")
    for i in range(min(5, len(df))):
        print(f"\nGốc: {df['sentence'].iloc[i]}")
        print(f"Sau xử lý: {df['processed_text'].iloc[i]}")

print("Hoàn thành xử lý tất cả tập dữ liệu!")


### TRÍCH XUẤT ĐẶC TRƯNG = 3 PHƯƠNG PHÁP TDIDF, WORD2VEC(CBOW,SKIP-GRAM)

In [ ]:
import pandas as pd
import os
# Đường dẫn tới thư mục chứa các tệp CSV đã xử lý
processed_data_dir = "processed_data"

# Đọc các tệp CSV đã xử lý
df_train = pd.read_csv(os.path.join(processed_data_dir, "train_processed.csv"))
df_val = pd.read_csv(os.path.join(processed_data_dir, "val_processed.csv"))
df_test = pd.read_csv(os.path.join(processed_data_dir, "test_processed.csv"))

y = df_train['sentiment'].values

In [ ]:
# -------------------- Trích xuất đặc trưng TF-IDF --------------------
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df_train['processed_text'])
joblib.dump(tfidf_vectorizer, os.path.join(model_dir, "TF-IDF_vectorizer.pkl"))

['saved_models\\TF-IDF_vectorizer.pkl']

In [ ]:

# -------------------- Trích xuất đặc trưng = word2vec 2 pp (CBOW & Skip-gram) --------------------
sentences = [text.split() for text in df_train['processed_text']]

In [ ]:
# Train Word2Vec models
w2v_cbow = Word2Vec(sentences, vector_size=100, window=5, sg=0, min_count=2, workers=multiprocessing.cpu_count())
w2v_sg = Word2Vec(sentences, vector_size=100, window=5, sg=1, min_count=2, workers=multiprocessing.cpu_count())

2025-02-19 15:05:26,436 - INFO - collecting all words and their counts
2025-02-19 15:05:26,436 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-02-19 15:05:26,453 - INFO - PROGRESS: at sentence #10000, processed 95493 words, keeping 3820 word types
2025-02-19 15:05:26,472 - INFO - collected 6163 word types from a corpus of 201166 raw words and 19569 sentences
2025-02-19 15:05:26,472 - INFO - Creating a fresh vocabulary
2025-02-19 15:05:26,485 - INFO - Word2Vec lifecycle event {'msg': 'effective_min_count=2 retains 3145 unique words (51.03% of original 6163, drops 3018)', 'datetime': '2025-02-19T15:05:26.485344', 'gensim': '4.3.3', 'python': '3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'prepare_vocab'}
2025-02-19 15:05:26,485 - INFO - Word2Vec lifecycle event {'msg': 'effective_min_count=2 leaves 198148 word corpus (98.50% of original 201166, drops 3018)',

In [ ]:
w2v_sg.save("saved_models/word2vec_skipgram.model")


2025-02-19 15:51:52,682 - INFO - Word2Vec lifecycle event {'fname_or_handle': 'saved_models/word2vec_skipgram.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-02-19T15:51:52.682858', 'gensim': '4.3.3', 'python': '3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'saving'}
2025-02-19 15:51:52,683 - INFO - not storing attribute cum_table
2025-02-19 15:51:52,689 - INFO - saved saved_models/word2vec_skipgram.model


In [ ]:
# Function to convert sentences to vectors using a trained Word2Vec model
def sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Convert all sentences to vectors using CBOW and Skip-gram
X_w2v_cbow = np.array([sentence_vector(text, w2v_cbow) for text in df_train['processed_text']])
X_w2v_sg = np.array([sentence_vector(text, w2v_sg) for text in df_train['processed_text']])

In [ ]:
# -------------------- Define models --------------------
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "MLP Classifier": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, activation='relu', solver='adam'),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC(kernel='linear', probability=True)
}

In [ ]:
# -------------------- Train & Evaluate Models --------------------
import joblib
import os

results = []
model_dir = "saved_models"
os.makedirs(model_dir, exist_ok=True)

for feature_name, X in [("TF-IDF", X_tfidf), ("Word2Vec CBOW", X_w2v_cbow), ("Word2Vec Skip-gram", X_w2v_sg)]:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for model_name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

        # save results
        results.append({
            "Feature": feature_name,
            "Model": model_name,
            "Accuracy": accuracy,
            "F1 Score": f1,
            "Precision": precision,
            "Recall": recall,
            "Training Time (s)": train_time
        })
        model_filename = f"{model_dir}/{feature_name}_{model_name}.pkl"
        joblib.dump(model, model_filename)

d:\anaconda\envs\aka_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("results.csv", index=False)


In [ ]:
results_df

,Feature,Model,Accuracy,F1 Score,Precision,Recall,Training Time (s)
0,TF-IDF,Logistic Regression,0.834440,0.824385,0.824309,0.834440,0.399071
1,TF-IDF,MLP Classifier,0.817833,0.817254,0.816707,0.817833,311.664151
2,TF-IDF,Random Forest,0.833163,0.818536,0.830403,0.833163,17.189322
3,TF-IDF,SVM,0.836995,0.825952,0.826777,0.836995,51.411999
4,Word2Vec CBOW,Logistic Regression,0.781809,0.757564,0.771994,0.781809,0.665790
5,Word2Vec CBOW,MLP Classifier,0.811191,0.801486,0.800449,0.811191,20.208349
6,Word2Vec CBOW,Random Forest,0.780020,0.763212,0.773511,0.780020,15.893234
7,Word2Vec CBOW,SVM,0.768779,0.720087,0.797292,0.768779,48.182016
8,Word2Vec Skip-gram,Logistic Regression,0.800204,0.779410,0.787791,0.800204,0.458314
9,Word2Vec Skip-gram,MLP Classifier,0.833418,0.825379,0.825073,0.833418,17.331103


In [ ]:
import joblib
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

# Định nghĩa đường dẫn đến mô hình đã lưu
model_dir = "saved_models"

# Load TF-IDF vectorizer & Logistic Regression model
tfidf_vectorizer = joblib.load(f"{model_dir}/TF-IDF_vectorizer.pkl")
logistic_model = joblib.load(f"{model_dir}/TF-IDF_Logistic Regression.pkl")

# Load Word2Vec Skip-gram & MLP Classifier
w2v_sg = Word2Vec.load(f"{model_dir}/word2vec_skipgram.model") 
mlp_model = joblib.load(f"{model_dir}/Word2Vec Skip-gram_MLP Classifier.pkl")

# Hàm chuyển câu thành vector với Word2Vec Skip-gram
def sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Hàm để dự đoán sentiment từ input
def predict_sentiment(input_text):
    # TF-IDF + Logistic Regression
    tfidf_features = tfidf_vectorizer.transform([input_text])
    tfidf_prediction = logistic_model.predict(tfidf_features)[0]

    # Word2Vec Skip-gram + MLP Classifier
    w2v_features = sentence_vector(input_text, w2v_sg).reshape(1, -1)
    w2v_prediction = mlp_model.predict(w2v_features)[0]

    return {
        "TF-IDF + Logistic Regression": tfidf_prediction,
        "Word2Vec Skip-gram + MLP": w2v_prediction
    }

# Test câu đầu vào
# input_sentence = "Cái phòng ctsv rất tích cực tổ chức sự kiện cho sinh viên tham gia tớ rất thích á"
input_sentence = "Giảng dạy bố láo cướp tiền là giỏi"
predictions = predict_sentiment(input_sentence)

print("🔹 Kết quả dự đoán:")
sentiment_mapping = {0: "tiêu cực", 1: "bình thường", 2: "tích cực"}
mapped_predictions = {model: sentiment_mapping[pred] for model, pred in predictions.items()}
print(mapped_predictions)


🔹 Kết quả dự đoán:
{'TF-IDF + Logistic Regression': 'tích cực', 'Word2Vec Skip-gram + MLP': 'tích cực'}
